<h2>Introduction</h2>

Here you will find a step by step guide to downloading, configuring, and running the Einstein Toolkit. You may use this tutorial on a workstation or laptop, or on a supported cluster. Configuring the Einstein Toolkit on an unsupported cluster is beyond the scope of this tutorial. If you find something that does not work, please feel free to email users@einsteintoolkit.org.

This tutorial is very basic and does not describe the internal workings of the Einstein Toolkit. For a more detailed introduction, please have a look a the [text](https://arxiv.org/abs/1305.5299) provided by Miguel Zilhão and Frank Löffler and the [one](https://arxiv.org/abs/2011.13314) by Nicholas Choustikov.

<h2>Prerequisites</h2>
When using the Einstein Toolkit on a laptop or workstation you will want a number of packages installed in order to download, compile and use the Einstein Toolkit components. If this is a machine which you control (i.e. you have root), you can install using one of the recipes that follow:

On Mac, please first 
- Install [Xcode](https://itunes.apple.com/us/app/xcode/id497799835) from the Apple [App Store](https://itunes.apple.com/us/app/xcode/id497799835). In *addition* agree to Xcode license and install the Xcode Command Line Tools in Terminal 
```bash
sudo xcodebuild -license
sudo xcode-select --install
```
- when using MacPorts
  - install MacPorts for your version of the Mac operating system, if you have not already installed it (https://www.macports.org/install.php). 
  - Next, please install the following packages, using the commands:
```bash
sudo port -N install pkgconfig gcc12 openmpi-gcc12 fftw-3 gsl zlib openssl subversion ld64 hdf5 +fortran +gfortran
sudo port select mpi openmpi-gcc12-fortran
sudo port select gcc mp-gcc12
```
- when using HomeBrew
  - install HomeBrew for your version of the Mac operating system, if you have not already installed it (https://brew.sh/). 
  - make sure to add /opt/homebrew/bin to your PATH as instructed by the installation script. You may need to restart your terminal. 
  - Next, please install the following packages, using the commands:
```bash
brew install fftw gcc gsl hdf5 hwloc jpeg open-mpi openssl zlib pkg-config bash subversion
brew link --force jpeg openssl zlib
```

On Debian/Ubuntu/Mint use this command (the strange syntax is to suport all three of them):
```bash
$(sudo -l sudo) su -c 'apt-get update'
$(sudo -l sudo) su -c 'apt-get install -y subversion gcc git numactl libgsl-dev libpapi-dev python3 python-is-python3 python3-pip libhwloc-dev libudev-dev make libopenmpi-dev libhdf5-openmpi-dev libfftw3-dev libssl-dev liblapack-dev g++ curl gfortran patch pkg-config libhdf5-dev libjpeg-turbo?-dev'
```

On Fedora use this command:
```bash
sudo dnf install -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which papi-devel perl python3 python3-pip hwloc-devel openmpi-devel hdf5-openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran findutils hdf5-devel fftw-devel patch gsl-devel pkgconfig
module load mpi/openmpi-x86_64
```
You will have to repeat the `module load` command once in each new shell each time you would like to compile or run the code. You may have to log out and back in for the module command to become visible.

On Centos use this command:
```bash
su -c 'yum install -y epel-release'
su -c 'yum install --enablerepo=crb -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which python3 python3-pip papi-devel hwloc-devel openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran fftw-devel patch gsl-devel perl'
module load mpi/openmpi-x86_64
```
You will have to repeat the `module load` command once in each new shell each time you would like to compile or run the code. You may have to log out and back in for the module command to become visible.

On OpenSuse use this command:
```bash
sudo zypper install -y curl gcc git lapack-devel make subversion gcc-c++ which python3 python3-pip papi-devel hwloc-devel openmpi-devel libopenssl-devel libnuma-devel gcc-fortran hdf5-devel libfftw3-3 patch gsl-devel pkg-config
mpi-selector --set  $(mpi-selector --list | head -n1)
```
You will only have to execute the `mpi-selector` once, after that log out and back in to make the `mpirun` and `mpicc` commands visible without which Cactus will compile very slowly and fail to run.

On Windows 10/11 please install the Ubuntu Linux subsystem, then follow the instructions for a Ubuntu system. [These](https://docs.microsoft.com/en-us/windows/wsl/install) are Microsoft's official instructions on how to do so, [Ubuntu](https://ubuntu.com/wsl#install-ubuntu-on-wsl) provides an alternative version. You may also want to install native ssh client like [Putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/) and an X11 server like [VcXsrv](https://sourceforge.net/projects/vcxsrv/), [XMing](https://sourceforge.net/projects/xming/) or an all-in-one solution for X11 server and ssh client like [MobaXterm](https://mobaxterm.mobatek.net/).

## Notebook setup
This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document, as a jupyter notebook that you can download and also in your own docker container using `nds-org/jupyter-et`. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [ ]:
# %load /usr/local/lib/et-setup.py
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
from time import time, sleep
import os
home = os.path.expanduser("~/")
@register_cell_magic
def bash(line, cell):
    get_ipython().system(cell)
@register_cell_magic
def slurm(line, cell):
    with open(f"{home}/.slurm.sh","w") as fd:
        fd.write("if [ ${SLURM_PROCID} != 0 ]; then exit 0; fi\n")
        fd.write(cell)
    get_ipython().system(f"srun {line} bash {home}/.slurm.sh")

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# We are going to install kuibit, a Python package to post-process Cactus simulations.
# We will install kuibit inside the Cactus directory. The main reason for this is to
# have a make easier to uninstall kuibit (you can just remove the Cactus folder). 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")


<b>Note:</b> By default, the cells in this notebook are Python commands. However, cells that start with <code>%%bash</code> are executed in a bash shell. If you wish to run these commands outside the notebook and in a bash shell, cut and paste only the part after the initial <code>%%bash</code>. 

## Getting Help
Gitter, email, bug trackers, etc can all be found here https://einsteintoolkit.org/support.html

## Optimized Download/Build Experience
Downloading the source code from github in a classroom setting, where lots of users are doing the same thing at the same time, can create network problems, and compiling the complete ET from scratch can take up to half an hour.

The next cell will create a complete pre-built ET checkout in your home directory, speeding up subsequent cells.  This step is optional, but should allow you to execute the notebook in less time.

**Note:** This will only work in the docker image or on the tutorial server.

In [ ]:
%%bash
cd ~/
untar.py ~etuser/Cactus.tar.gz

<h2>Download</h2>

A script called GetComponents is used to fetch the components of the Einstein Toolkit. GetComponents serves as convenient wrapper around lower level tools like git and svn to download the codes that make up the Einstein toolkit from their individual repositories. You may download and make it executable as follows:

In [ ]:
cd ~/

In [ ]:
%%bash
curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2023_05/GetComponents
chmod a+x GetComponents

GetComponents accepts a thorn list as an argument. To check out the needed components:

In [ ]:
%%bash
./GetComponents https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2023_05/einsteintoolkit.th

In [ ]:
cd ~/Cactus

<h2>Configure and build</h2>

The recommended way to compile the Einstein Toolkit is to use the Simulation Factory ("SimFactory").
<h3>Configuring SimFactory for your machine</h3>

The ET depends on various libraries, and needs to interact with machine-specific queueing systems and MPI implementations. As such, it needs to be configured for a given machine. For this, it uses SimFactory. Generally, configuring SimFactory means providing an optionlist, for specifying library locations and build options, a submit script for using the batch queueing system, and a runscript, for specifying how Cactus should be run, e.g. which mpirun command to use.

In [ ]:
%%bash
./simfactory/bin/sim setup-silent

After this step is complete you will find your machine's default setup under ./simfactory/mdb/machines/&lt;hostname &gt;.ini
You can edit some of these settings freely, such as "description", "basedir" etc. Some entry edits could result in simulation start-up warnings and/or errors such as "ppn" (processor-per-node meaning number of cores on your machine), "num-threads" (number of threads per core) so such edits must be done with some care.

<h2>Building the Einstein Toolkit</h2>

Assuming that SimFactory has been successfully set up on your machine, you should be able to build the Einstein Toolkit with the command below. The option "-j2" sets the make command that will be used by the script. The number used is the number of processes used when building. Even in parallel, this step may take a while, as it compiles all the thorns specified in thornlists/einsteintoolkit.th.

Note: Using too many processes to compile on the test machine may result in compiler failures, if the system runs out of memory.

In [ ]:
%%slurm -n 2
./simfactory/bin/sim build -j${SLURM_NPROCS} --thornlist thornlists/einsteintoolkit.th

<h2>Running a simple example</h2>

You can now run the Einstein Toolkit with a simple test parameter file.

In [ ]:
%%bash
./simfactory/bin/sim create-run helloworld \
    --parfile arrangements/CactusExamples/HelloWorld/par/HelloWorld.par

The above command will run the simulation naming it "helloworld" and display its log output to screen.

If you see <pre>INFO (HelloWorld): Hello World!</pre> anywhere in the above output, then congratulations, you have successfully downloaded, compiled and run the Einstein Toolkit! You may now want to try some of the other tutorials to explore some interesting physics examples.

## Running a Simple Wave Equation
The wave equation can be described in Cartesian coordinates by the equations
$u_{,t} = \rho \\
\rho_{,t} = c^2 \left(u_{,xx} + u_{,yy} \right)$
or, in spherical symmetry, it can be written as
$u_{,t} = \rho \\
\rho_{,t} = c^2 \frac{1}{r^2}\left( r^2 u_{,r}\right)_{,r}$
using polar coordinates. This code is implemented using Kranc (a Mathematica tool that generates Cactus code).

In [ ]:
%%writefile repos/mclachlan/ML_WaveToy_Test/test/gaussian-RK4.par
# gaussian-RK4.par
# Evolve the scalar wave equation with the RK4 integrator

ActiveThorns = "
   Boundary
   Carpet
   CarpetIOASCII
   CarpetIOBasic
   CarpetIOScalar
   CarpetLib
   CarpetReduce
   CartGrid3D
   CoordBase
   GenericFD
   IOUtil
   LoopControl
   ML_WaveToy
   MoL
   SymBase
   Time
"



Carpet::domain_from_coordbase = yes
CartGrid3D::type              = "coordbase"

CoordBase::domainsize = "minmax"
CoordBase::spacing    = "numcells"
CoordBase::xmin       = -5.0
CoordBase::ymin       = -5.0
CoordBase::zmin       = -5.0
CoordBase::xmax       = +5.0
CoordBase::ymax       = +5.0
CoordBase::zmax       = +5.0
CoordBase::ncells_x   = 50
CoordBase::ncells_y   = 50
CoordBase::ncells_z   = 50

CoordBase::boundary_size_x_lower = 2
CoordBase::boundary_size_y_lower = 2
CoordBase::boundary_size_z_lower = 2
CoordBase::boundary_size_x_upper = 2
CoordBase::boundary_size_y_upper = 2
CoordBase::boundary_size_z_upper = 2
Carpet::ghost_size               = 2



Cactus::cctk_itlast = 100

MoL::ODE_method             = "RK4"
MoL::MoL_Intermediate_Steps = 4
MoL::MoL_Num_Scratch_Levels = 1

Time::dtfac = 0.5



ML_WaveToy::initial_data = "Gaussian"
ML_WaveToy::WT_u_bound   = "newrad"
ML_WaveToy::WT_rho_bound = "newrad"



IO::out_dir      = $parfile
#IO::out_fileinfo = "none"

IOBasic::outInfo_every = 1
IOBasic::outInfo_vars  = "ML_WaveToy::u"

IOScalar::outScalar_reductions = "norm1 norm2 minimum maximum norm_inf"
IOScalar::outScalar_every      = 1
IOScalar::outScalar_vars       = "ML_WaveToy::WT_u"

IOASCII::out1D_every = 1
IOASCII::out1D_vars  = "ML_WaveToy::WT_u ML_WaveToy::WT_rho ML_WaveToy::WT_eps"

CarpetIOASCII::compact_format = yes
CarpetIOASCII::output_ghost_points = no

In [ ]:
%%bash
./simfactory/bin/sim create-submit wavetoy --cores 2 \
    --parfile repos/mclachlan/ML_WaveToy_Test/test/gaussian-RK4.par

In [ ]:
%%bash
OUTPUT_DIR=$(./simfactory/bin/sim get-output-dir wavetoy)
echo "The output directory is:"
ls -d $OUTPUT_DIR/gaussian-RK4
echo
echo "The contents of the output directory is:"
ls $OUTPUT_DIR/gaussian-RK4

### Plotting the output
The output for this wavetoy consists of simple ascii files. The rho function (which maps to either u or v in the equations above), is shown in the next cell. The "d" is for diagonal.

In [ ]:
!head -20 $HOME/simulations/wavetoy/output-0000/gaussian-RK4/u.x.asc

In [ ]:
# Load the data using numpy
import os
import numpy as np
home = os.environ["HOME"]
data = np.genfromtxt(os.path.join(home,"simulations","wavetoy","output-0000","gaussian-RK4","u.x.asc"))
print(data.shape)

In [ ]:
# Load the timesteps from the output data
time_steps = np.unique(data[:,0])
print(time_steps)

In [ ]:
# Plot the first 10 timesteps
import matplotlib.pyplot as plt
x = data[data[:,0] == time_steps[0]][:,5]
for time_step in time_steps[:10]:
    step_data = data[data[:,0] == time_step][:,-1]
    plt.plot(x,step_data)

In [ ]:
# Create a movie using all the timesteps
import matplotlib.pyplot as plt
frameno = 0
for time_step in time_steps:
    plt.clf()
    step_data = data[data[:,0] == time_step][:,-1]
    plt.ylim([-1,1])
    plt.plot(step_data)
    frameno += 1
    plt.savefig("plot%03d.png" % frameno)
    plt.close()

from subprocess import call
call(["ffmpeg", "-i", "plot%03d.png", "-i", "plot001.png",
      "-filter_complex", "[1:v] palettegen [p];[0:v][p] paletteuse",
      "-y", "output.gif"])

In [ ]:
# Show the movie
from IPython.display import Image

Image("output.gif")

## Managing submitted simulations

Since the `submit` command was used to start the  simulation, it is running in the background and you have to use simfactory commands to interact with it. The next cell shows how to list simulations.

**Remember** that you have to interrupt the kernel to stop `show-output` and be able to execute the cells below.

In [ ]:
%%bash
./simfactory/bin/sim list-simulations

Simfactory offers a `stop` command to abort a running simulation. The next cell has the command intentionally commented out to prevent accidental stopping of your very first simulation.

In [ ]:
%%bash
#./simfactory/bin/sim stop wavetoy

after this the simulation changes to the "FINISHED" state indicating it is no longer running.

Simulations that are no longer needed are removed using the `purge` command. The next cell has the command intentionally commented out to prevent accidental removing of your very first simulation.

In [ ]:
%%bash
#./simfactory/bin/sim purge wavetoy

In [ ]:
%%bash
./simfactory/bin/sim show-output wavetoy --follow